# Setup

In [ ]:
from parameters.parameters import prune_quantiles, prune_measure

debug = True # If True, will produce plots and/or verbose output to double-check

import sys
sys.path.append("..")

# Path
print("Current working directory: ", sys.path[0])
%run -i "../parameters/parameters.py"
%run -i "../src/path.py"
%run -i "../src/setup.py"
%run -i "../src/functions.py"
%run -i "greedy_triangulation.py"
%run -i "graph_similarity.py"

%load_ext watermark
%watermark -n -v -m -g -iv

# Run the program

In [ ]:
for placeid, placeinfo in tqdm(cities.items(), desc="Cities"):
    print(placeid + ": Generating graph and POIs")

    G_carall = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'carall')
    with open(PATH["data"] + placeid + "/" + placeid + '_poi_' + poi_source + '_nnidscarall.csv') as f:
        nnids = [int(line.rstrip()) for line in f]

    subgraph_percentage = 0.6
    prune_quantiles = [0.5]
    for prune_measure in ["betweenness", "closeness", "random"]:
        print(f"Prune measure: {prune_measure}")
        data = [
            "sum_of_errors",
            "continuous_vertices",
            "bikengrowth_vertices",
            "continuous_disconnected_points",
            "bikengrowth_disconnected_points",
        ]

        (GTs, GT_abstracts) = _greedy_triangulation_routing(G_carall, nnids, prune_quantiles, prune_measure)

        for i in range(100):
            if (i + 1) % 25 == 0:
                print(f"Running iteration {i + 1}")
            if prune_measure == "random":
                (GTs, GT_abstracts) = _greedy_triangulation_routing(G_carall, nnids, prune_quantiles, prune_measure)

            gts, abstract_gts = greedy_triangulation_in_steps(G_carall, nnids, subgraph_percentage, prune_quantiles, prune_measure)

            for prune_quantile, gt, szell_gt in zip(prune_quantiles, gts, GTs):
                error, our_disconnected_points, szell_disconnected_points = sum_of_errors_pois(gt, szell_gt, nnids)
                data.append([error, gt.vcount(), szell_gt.vcount(), our_disconnected_points, szell_disconnected_points])

        print(f"Writing data to {placeid}_{prune_measure}.csv")
        with open(f'{placeid}_{prune_measure}.csv', 'w') as f:
            writer = csv.writer(f)
            writer.writerows(data)

In [ ]:
for placeid, placeinfo in tqdm(cities.items(), desc="Cities"):
    result_file = open(PATH["results"] + placeid + "/" + filename + ".pickle", 'rb')
    res = pickle.load(result_file)
    szell_gts = res["GTs"]